# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [1]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [2]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [3]:
%store -r numerosKalaba typeKalaba
numerosKalaba=[5]
# numerosKalaba=[18]
print numerosKalaba
typeKalaba="Kalaba"
%store numerosKalaba 
%store typeKalaba

[5]
Stored 'numerosKalaba' (list)
Stored 'typeKalaba' (str)


In [4]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+"23-"
    nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
else:
    repertoire=home+"/Library/Mobile Documents/com~apple~CloudDocs/Downloads/"
    serie=repertoire+"23-Kanoniks/"
    nomsKalabas=[serie+"Kanonik-%02d-ok/"%num for num in numerosKalaba]

# Traitement

In [5]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [6]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    
    lexiqueTestPrep=getExemple(stems["PREP"]).lower()
    lexiqueTestNoun=getExemple(stems["NOM"]).lower()
    lexiqueTestHyper=getExemple(stems["ADJ"]).lower()
    
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    # classesNom=PFM.lexique.formeLexeme[mot.lower()][0].split('.')[1:]
    # [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/23-K5/
NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre Cas
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B
head stems,NOM,D
head stems,VER
head stems,VER,VI
head stems,VER,VT
head stems,ADJ


### Bilan

In [7]:
stems["NOM"]

{'A': {'N1': {'ZeweZi': ['raison', 'raisons'],
   'gidon': [u'obscurit\xe9', u'obscurit\xe9s'],
   'puZis': [u'col\xe8re', u'col\xe8res'],
   'rodeg': [u'h\xe9ros', u'h\xe9roS', u'h\xe9ro\xefnes', u'h\xe9ro\xefne'],
   'sogod': ['croc', 'crocs'],
   'toTug': ['chemin', 'chemins'],
   'vjolet': ['Violette', 'Violettes']},
  'N2': {'bufeS': ['table', 'tables'],
   'fuZus': ['fille', 'filles'],
   'janof': ['bras', 'braS'],
   'ledaf': ['homme', 'hommes'],
   'nasip': ['ombre', 'ombres'],
   'nuDuN': ['sorcier', u'sorci\xe8re', 'sorciers', u'sorci\xe8res'],
   'pazet': ['couteau', 'couteaux'],
   'siSal': [u'for\xeat', u'for\xeats'],
   'vetot': [u'caf\xe9', u'caf\xe9s']},
  'N3': {'dupad': ['gorge', 'gorges'],
   'fadij': ['histoire', 'histoires'],
   'gozil': [u'p\xe8re', u'p\xe8res'],
   'katiSa': ['Katisha', 'Katishas'],
   'kilit': ['louve', 'louves', 'lOUP', 'lOUPs'],
   'wewir': [u'ma\xeetre', u'ma\xeetresses', u'ma\xeetresse', u'ma\xeetres'],
   'witov': ['chambre', 'chambres']},


In [8]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
# print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

gros.A1
après


In [9]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=A',
 u'VER, Temps=PRS, Pers=3Sg, Genre=B',
 u'VER, Temps=PRS, Pers=3Sg, Genre=C',
 u'VER, Temps=PRS, Pers=3Sg, Genre=D',
 u'VER, Temps=PRS, Pers=3Pau, Genre=A',
 u'VER, Temps=PRS, Pers=3Pau, Genre=B',
 u'VER, Temps=PRS, Pers=3Pau, Genre=C',
 u'VER, Temps=PRS, Pers=3Pau, Genre=D',
 u'VER, Temps=PRS, Pers=3Pl, Genre=A',
 u'VER, Temps=PRS, Pers=3Pl, Genre=B',
 u'VER, Temps=PRS, Pers=3Pl, Genre=C',
 u'VER, Temps=PRS, Pers=3Pl, Genre=D',
 u'VER, Temps=PST, Pers=3Sg, Genre=A',
 u'VER, Temps=PST, Pers=3Sg, Genre=B',
 u'VER, Temps=PST, Pers=3Sg, Genre=C',
 u'VER, Temps=PST, Pers=3Sg, Genre=D',
 u'VER, Temps=PST, Pers=3Pau, Genre=A',
 u'VER, Temps=PST, Pers=3Pau, Genre=B',
 u'VER, Temps=PST, Pers=3Pau, Genre=C',
 u'VER, Temps=PST, Pers=3Pau, Genre=D',
 u'VER, Temps=PST, Pers=3Pl, Genre=A',
 u'VER, Temps=PST, Pers=3Pl, Genre=B',
 u'VER, Temps=PST, Pers=3Pl, Genre=C',
 u'VER, Temps=PST, Pers=3Pl, Genre=D']

In [10]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [11]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[('X+al', 'CF=V1, Temps=PRS')]

In [12]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [13]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

fuir.VT.V1 Ziwaz
échanger.VT.V1 kokek
dernier.A1 gelom
entrer.VI.V1 rakez
placer.VT.V1 puzem
tourner-Mov.VI.V2 rolan
transpercer.VT.V1 fudat
courageux.A2 potoZ
mort.B.N4 wezaf
légume.C.N4 Semot
demande.B.N1 putis
changer.VT.V2 bazum
apporter.VT.V1 duZut
retourner.VI.V2 TiTaS
manoir.B.N4 bunod
oeil.C.N3 geret
jeter.VT.V2 NiNut
viande.D.N3 dakuD
furieux.A1 TiZiZ
nourrir.VT.V1 dinak
vote.B.N4 DeZiS
présenter.VT.V2 zuvod
Hoderi.B.N4 oderi
trois.A2 ruNos
pendre.VT.V1 Dugab
demander.VT.V2 vumot
sorcier.A.N2 nuDuN
tornade.A.N4 gunus
reconnaître.VT.V1 Senap
mère.A.N4 josus
légende.C.N2 guniN
sur dif
courir.VI.V1 Dozol
villageois.C.N4 nepas
planter.VT.V1 luzaf
envahir.VT.V2 bemes
maison.C.N2 geTep
voler.VT.V1 saZun
effrayé.A2 rivek
construction.D.N4 buguN
troisième.A2 sisiSu
deux.A2 roNoN
œuf.B.N3 jogus
peau.B.N2 kakip
main.D.N2 belof
marais.A.N4 nebap
connaître.VT.V1 nuziS
jour.D.N2 Satud
infirmière.B.N1 bodup
PRO deZ
compter.VT.V2 govet
cinq.A1 Nulov
de wu
devant bizoT
visage.D.N1 Sapis
femme

# Conclusion

In [14]:
ding()